# MPC


In [215]:
import cvxpy as cp
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

dt = 0.1
start = 0 
stop = 10 
dummyDataX = np.arange(start ,stop ,dt)
dummyDataY = np.ones(len(dummyDataX))

def vel(begin , end ): 
    return (end - begin)/dt

def plot(absTrajectory , calTrajectory):
    fig, axs = plt.subplots(2)
    fig.suptitle('Vertically stacked subplots')
    axs[0].plot(absTrajectory[0], absTrajectory[1])
    axs[0].plot(calTrajectory[0], calTrajectory[1])
    
    axs[1].plot(absTrajectory[0], absTrajectory[1])

    plt.show()

In [218]:
class vehicle_dynamics:

    def __init__(self, dt):
        
#       Exercise 1: Enter the correct state space matrices using these statements, replacing "0." where necessary
#       Uncomment the code below and remove the "NotImplementedError"

        self.A_c = np.array([
            [0, 1],
            [0., 0]])

        self.B_c = np.array([
            [1],
            [1]])

        self.C_c = np.array([
            [1, 0]])

        self.D_c = np.array([
            [0.]])
        
    
        # Euler discretization
        self.A = np.eye(2) + self.A_c * dt
        self.B = self.B_c * dt
        self.C = self.C_c
        self.D = self.D_c

        # Limits on state and input
        self.x_min = np.array([-1., -10])
        self.x_max = np.array([100, 25])
        self.u_min = -4.0
        self.u_max = 4.0
        
        # Used later
        self.theta_min = 0
        self.theta_max = 0
        
    def next_x(self, x, u):
        return self.A.dot(x) + self.B.transpose().dot(u)
 


def mpc_control(vehicle, N, x_init, x_target):
    # DO NOT CHANGE THESE PARAMETERS #
    weight_input = 0.2*np.eye(1)    # Weight on the input
    weight_tracking = np.array([[1,0],[0,0],[0,0]]) # Weight on the tracking state
    
    cost = 0.
    constraints = []
    
    # Create the optimization variables
    x = cp.Variable((2, N + 1)) # cp.Variable((dim_1, dim_2))
    u = cp.Variable((1, N))
    
    constraints += [x[:, 0] == x_init]
        
    for k in range(N):
        # State space
        state_ = vehicle.A@x[:,k]
        input_ = vehicle.B@u[:, k]
      
        
        # constraints
        constraints += [x[:, k+1] == state_ + input_]

        # Minimize the cost function
        cost += cp.quad_form(u[:, k], weight_input)
        cost += cp.quad_form((x_target - x[:, k+1] ), weight_tracking)
 
    # Solves the problem
    problem = cp.Problem(cp.Minimize(cost), constraints)
    problem.solve(solver=cp.OSQP)

    # We return the MPC input and the next state (and also the plan for visualization)
    return u[:, 0].value, x[:, 1].value, x[:, :].value, None

lookahead = 10
state_ = np.array([0,0])
input_ = np.array([0,0])
vehicle = vehicle_dynamics(dt)
target = np.zeros((2,2)) 

# Saving data 
TakingInputs = []
NextState = []
error = []

for i in tqdm(range(len(dummyDataX))):
    
    print(f"this is the state: {state_}")
    
    # Get the next state
    state_ = vehicle.next_x(state_ , input_)
    
    # Reshaping data
    X = np.reshape(dummyDataX, (dummyDataX.shape[0], 1))
    Y = np.reshape(dummyDataY, (dummyDataY.shape[0], 1))
    target = np.concatenate((X,Y), axis= 1)
    
    # Calculate control input
    input_   = mpc_control(vehicle, 10, state_, target[i,:])
    
    # Adding zero to the end to be compliant
    input_ = np.pad(input_[0], (0, 1), 'constant')

    #Append to the lists
    TakingInputs.append(input_)
    NextState.append(state_)
    error.append(state_ - target[i,:] )
    

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

this is the state: [0 0]


Exception: Invalid dimensions for arguments.

In [214]:

class unicycle_dynamics:

    def __init__(self, dt, theta):
        
        self.theta = theta
        self.x = np.array([[0],[0],[0]], dtype = np.float64) 
        
        self.A_c = np.array([
            [np.cos(self.theta), 0, 0],
            [0 , np.sin(self.theta), 0],
            [0,0, 1]])

    def next_x(self, u):
        nextIteration = self.A_c.dot(u)
        self.x += nextIteration*dt
        self.theta = nextIteration[2]
        return nextIteration 
    

In [212]:
def mpc_control(vehicle, N, x_init, x_target):
    # DO NOT CHANGE THESE PARAMETERS #
    weight_input = np.array([[1,0,0],[0,1,0],[0,0,0]])    # Weight on the input
    weight_tracking = np.array([[1,0,0],[0,1,0],[0,0,0]]) # Weight on the tracking state
    
    cost = 0.
    constraints = []
    
    # Create the optimization variables
    x = cp.Variable((3, N + 1)) # cp.Variable((dim_1, dim_2))
    u = cp.Variable((3, N))
    
    constraints += [x[:, 0] == x_init]
        
    for k in range(N):
        # State space
        state_ = vehicle.A@x[:,k]
      
        
        # constraints
        constraints += [x[:, k+1] == state_ + input_]

        # Minimize the cost function
        cost += cp.quad_form(u[:, k], weight_input)
        cost += cp.quad_form((x_target - x[:, k+1] ), weight_tracking)
 
    # Solves the problem
    problem = cp.Problem(cp.Minimize(cost), constraints)
    problem.solve(solver=cp.OSQP)

    # We return the MPC input and the next state (and also the plan for visualization)
    return u[:, 0].value, x[:, 1].value, x[:, :].value, None




In [208]:
from tqdm import tqdm

lookahead = 10
state_ = np.array([0,0])
input_ = np.array([0,0])
vehicle = vehicle_dynamics(dt)
target = np.zeros((2,2)) 

# Saving data 
TakingInputs = []
NextState = []
error = []

for i in tqdm(range(len(dummyDataX))):
    
    # Get the next state
    state_ = vehicle.next_x(state_ , input_)
    
    # Reshaping data
    X = np.reshape(dummyDataX, (dummyDataX.shape[0], 1))
    Y = np.reshape(dummyDataY, (dummyDataY.shape[0], 1))
    target = np.concatenate((X,Y), axis= 1)
    
    # Calculate control input
    input_   = mpc_control(vehicle, 10, state_, target[i,:])
    
    # Adding zero to the end to be compliant
    input_ = np.pad(input_[0], (0, 1), 'constant')

    #Append to the lists
    TakingInputs.append(input_)
    NextState.append(state_)
    error.append(state_ - target[i,:] )


  0%|                                                                                          | 0/100 [00:00<?, ?it/s]


ValueError: Cannot broadcast dimensions  (3,) (2,)